#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 5: Estimación de la demanda de bicicletas compartidas utilizando *Neural Networks*.  </center>

# Introducción

En esta actividad se retomará el problema de la competencia [*Bike Sharing Demand*](https://www.kaggle.com/c/bike-sharing-demand) visto en el Taller 3.
Esta vez las estimaciónes deben obtenerse utilizando la herramienta: *Multilayer Perceptron* (MLP). Es importante mantener la función *Root Mean Squared Logarithmic Error* (RMSLE) como medida de desempeño de manera de poder comparar los resultados con los obtenidos en el Taller 3.

Tanto las preguntas teóricas como la parte práctica de esta actividad están ligadas al contenido del capítulo 10 (*Introduction to
Artificial Neural Networks with
Keras*) del libro del curso.


## Objetivos


*   Trabajar con modelos MLP utilizando la librería [*Keras*](https://keras.io/api/).
*   Probar algunas de las herramientas disponibles para la busqueda de hiperparámetros.
*   Interpretar modelos con *Shapley Values*.

## Formas de trabajo

#### Opción 1: Trabajar localmente

##### Descarga de datos disponibles en Kaggle

Luego, para descargar el dataset de Demanda de Bicicletas:

In [2]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 980.3 kB/s eta 0:00:0031m2.1 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl.metadata (2.4 kB)
Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.6.12-py3-none-any.whl size=102971 sha256=6bffe181df099d8aec989185d65ac10feefcd7d99d3694e654a18fee1cbda944
  Stored in directory: /home/xqft/.cache/pip/wheels/f3/eb/e9/819c2d9eac90204eec8579430759f75a1d6dbe4cd0b93f53bc
Successfully built kaggle


In [3]:
!kaggle competitions download -c bike-sharing-demand

100%|█████████████████████████████████████████| 189k/189k [00:00<00:00, 431kB/s]
100%|█████████████████████████████████████████| 189k/189k [00:00<00:00, 431kB/s]


Descomprima el archivo descargado:

In [4]:
import shutil
shutil.unpack_archive('./bike-sharing-demand.zip', './')

#### Opción 2:  Trabajar en *Colab*. 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2024/blob/main/talleres/taller5_demanda_de_bicicletas_con_NNs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

Se puede trabajar en Google Colab. Para ello es necesario contar con una cuenta de **google drive** y ejecutar un notebook almacenado en dicha cuenta. De lo contrario, no se conservarán los cambios realizados en la sesión. En caso de ya contar con una cuenta, se puede abrir el notebook y luego ir a `Archivo-->Guardar una copia en drive`. 

La siguiente celda monta el disco personal del drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

A continuación, vaya a su cuenta de [Kaggle](https://www.kaggle.com/) (o cree una si aún no lo ha hecho), haga clic en el icono de perfil en la esquina superior derecha de la pantalla y seleccione "Your Account" en la lista desplegable. Luego, seleccione la viñeta "Account" y haga clic en "Create new API token". Entonces un archivo llamado kaggle.json se descargará automáticamente a su carpeta de descargas. Este archivo contiene sus credenciales de inicio de sesión para permitirle acceder a la API.

La siguiente celda realiza la configuración necesaria para obtener datos desde la plataforma Kaggle. Le solicitará que suba el archivo kaggle.json descargado anteriormente.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from google.colab import files

# El siguiente archivo solicitado es para habilitar la API de Kaggle en el entorno que está trabajando.
# Este archivo se descarga entrando a su perfíl de Kaggle, en la sección API, presionando donde dice: Create New API Token

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Una vez guardado el *token* se pueden descargar los datos, en este caso se bajarán los datos del dataset de Demanda de Bicicletas:

In [ ]:
!kaggle competitions download -c bike-sharing-demand

In [ ]:
!unzip bike-sharing-demand.zip

## Paquetes a utilizar

En esta actividad se utilizarán algunas bibliotecas auxiliares que deberán ser instaladas. Ejecutar la siguiente celda hasta que se ejecute sin errores. En caso de error, se puede instalar el paquete faltante desde el notebook con el comando:

`!pip install paquete_faltante`

In [1]:
#import comet_ml in the top of your file
#from comet_ml import Experiment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
#sns.set_theme(style="whitegrid")

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_submission = pd.read_csv('sampleSubmission.csv')

## Parte 1 - Procesamiento de los datos

Dado que ya se ha familiarizado con los datos, se implementa el mismo preprocesamiento que utilizó en el Taller 3.

In [2]:
df_train['datetime'] = pd.to_datetime(df_train['datetime'])

df_train['hour'] = df_train['datetime'].dt.hour
df_train['weekday'] = df_train['datetime'].dt.weekday
df_train['month'] = df_train['datetime'].dt.month 
df_train['year'] = df_train['datetime'].dt.year

y_train_full = df_train['count']
df_train = df_train.drop(['datetime', 'casual', 'registered', 'count'], axis=1) # hay que eliminarlas ya que tiene relación directa con la columna objetivo y no aparecen en el conjunto de *test*.

- Dado que se trabajará con redes neuronales, ¿Cree conveniente realizar alguna modificación en el preprocesamiento?

Se debe encodear los atributos categoricos y normalizar los numericos

## Parte 2 - Multilayer Perceptron (MLP)

Siguiendo el ejemplo de la sección *Building a Regression MLP Using the Sequential API*:


*   Implementar un estimador manteniendo los hiperparámetros del ejemplo.
*   ¿Cuál es la cantidad total de parámetros entrenables de la red?
*   Seleccionar aleatoriamente un *10%* de los datos para validación, y graficar la función de *loss* (*Mean Squared Logarithmic Error*) de entrenamiento y validación.

***Nota:** Observe que en el ejemplo se agrega una capa de normalización*

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_train, y_train_full, train_size=0.9, random_state=42)

In [7]:
X_train.shape[1:]

(12,)

In [93]:
y_train.shape

(9797,)

In [6]:
import tensorflow as tf

tf.keras.backend.clear_session()
tf.random.set_seed(42)

norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="msle", optimizer=optimizer, metrics=["MeanSquaredLogarithmicError"])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
#mse_test, rmse_test = model.evaluate(X_test, y_test)

Epoch 1/20
307/307 [==============================] - 2s 4ms/step - loss: 5.1012 - mean_squared_logarithmic_error: 5.1012 - val_loss: 1.3955 - val_mean_squared_logarithmic_error: 1.3955
Epoch 2/20
307/307 [==============================] - 1s 3ms/step - loss: 1.1244 - mean_squared_logarithmic_error: 1.1244 - val_loss: 0.9904 - val_mean_squared_logarithmic_error: 0.9904
Epoch 3/20
307/307 [==============================] - 1s 3ms/step - loss: 0.9455 - mean_squared_logarithmic_error: 0.9455 - val_loss: 0.8889 - val_mean_squared_logarithmic_error: 0.8889
Epoch 4/20
307/307 [==============================] - 1s 3ms/step - loss: 0.8656 - mean_squared_logarithmic_error: 0.8656 - val_loss: 0.8091 - val_mean_squared_logarithmic_error: 0.8091
Epoch 5/20
307/307 [==============================] - 1s 3ms/step - loss: 0.7877 - mean_squared_logarithmic_error: 0.7877 - val_loss: 0.7144 - val_mean_squared_logarithmic_error: 0.7144
Epoch 6/20
307/307 [==============================] - 1s 4ms/step - lo

## Parte 3 - Ajuste fino

Siguiendo el ejemplo de la sección *Fine-Tuning Neural Network Hyperparameters*:


*   Utilizar la herramienta *RandomSearch* de *KerasTuner* para la busqueda de hiperparámetros del modelo implementado en *keras*. 
*   Probar el *tip* que se sugiere en la sección *Number of Neurons per Hidden Layer* y comentar los resultados.

In [95]:
import keras_tuner as kt

norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=8)
    n_neurons = []
    for i in range(n_hidden):
        n_neurons.append(hp.Int(f"n_neurons{i}", min_value=16, max_value=50))
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log", default=0.0033138)
   
    model = tf.keras.Sequential()
    model.add(norm_layer)
    model.add(tf.keras.layers.Flatten())
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons[i], activation="relu"))
        
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss="msle", optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate), metrics=["msle"])
    
    return model       

In [26]:
random_search_tuner = kt.RandomSearch(
    build_model,
    objective="val_msle",
    max_trials=1000,
    overwrite=True,
    directory="nn_model",
    project_name="nn_model_rand_search",
    seed=42
)
random_search_tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Trial 1000 Complete [00h 00m 07s]
val_msle: 22.910757064819336

Best val_msle So Far: 0.542232871055603
Total elapsed time: 01h 57m 36s


In [13]:
random_search_tuner.oracle.get_best_trials(num_trials=1)[0]

## Parte 4 - Ajuste fino (Optuna)

*   Utilizar *Optuna* para la busqueda de hiperparámetros del modelo en *Keras*. Se le sugiere seguir uno de los siguientes ejemplos: [*keras_simple*](https://github.com/optuna/optuna-examples/blob/main/keras/keras_simple.py),  [OptunaSearchCV](https://github.com/optuna/optuna-examples/blob/main/sklearn/sklearn_optuna_search_cv_simple.py).

***Nota:** Optuna puede utilizarse para optimizar otras técnicas por fuera de las redes neuronales.*

***Nota2:** Keras Tuner permite realizar Optimización bayesiana.*

In [13]:
bayesian_tuner = kt.BayesianOptimization(
    build_model,
    objective="val_msle",
    max_trials=30,
    overwrite=True,
    directory="nn_model",
    project_name="nn_model_rand_search",
    seed=42
)
bayesian_tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Trial 30 Complete [00h 00m 05s]
val_msle: 1.1242798566818237

Best val_msle So Far: 0.899493396282196
Total elapsed time: 00h 02m 23s


In [6]:
#import comet_ml in the top of your file
#from comet_ml import Experiment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
#sns.set_theme(style="whitegrid")

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_submission = pd.read_csv('sampleSubmission.csv')

df_train['datetime'] = pd.to_datetime(df_train['datetime'])

df_train['hour'] = df_train['datetime'].dt.hour
df_train['weekday'] = df_train['datetime'].dt.weekday
#df_train['month'] = df_train['datetime'].dt.month 
df_train['year'] = df_train['datetime'].dt.year

y_train_full = df_train['count']
df_train = df_train.drop(['datetime', 'casual', 'registered', 'count'], axis=1) # hay que eliminarlas ya que tiene relación directa con la columna objetivo y no aparecen en el conjunto de *test*.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_train, y_train_full, train_size=0.9, random_state=42)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_train)
X_train = norm_layer(X_train)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_val)
X_val = norm_layer(X_val)

In [17]:
import keras_tuner as kt
import tensorflow as tf

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=8)
    n_neurons = []
    for i in range(n_hidden):
        n_neurons.append(hp.Int(f"n_neurons{i}", min_value=16, max_value=50, default=35))
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log", default=0.0033138)
   
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(n_neurons[i], activation="relu"))
        
    model.add(tf.keras.layers.Dense(1, activation="linear"))
    model.compile(loss="msle", optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate))
    
    return model

hyperband = kt.Hyperband(
    build_model,
    objective="val_loss",
    max_epochs=80,
    overwrite=True,
    directory="nn_model",
    project_name="nn_model_hyperband",
    seed=42
)

early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
hyperband.search(X_train, y_train, epochs=80, validation_data=(X_val, y_val), callbacks=[early])

Trial 40 Complete [00h 00m 02s]

Best val_loss So Far: 0.679317831993103
Total elapsed time: 00h 04m 03s

Search: Running Trial #41

Value             |Best Value So Far |Hyperparameter
4                 |7                 |n_hidden
19                |41                |n_neurons0
25                |36                |n_neurons1
47                |35                |n_neurons2
27                |19                |n_neurons3
28                |46                |n_neurons4
43                |38                |n_neurons5
32                |45                |n_neurons6
18                |49                |n_neurons7
0.0032988         |0.0046924         |learning_rate
9                 |3                 |tuner/epochs
3                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
1                 |0                 |tuner/round
0014              |None              |tuner/trial_id



Traceback (most recent call last):
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/xqft/.conda/envs/ml/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_k

In [9]:
best_model = hyperband.get_best_models()[0]
best_model.save("./best_hyperband")
#best_model = tf.keras.models.load_model('./best_hyperband')
best_model.evaluate(X_val, y_val)

ValueError: Model <keras.src.engine.sequential.Sequential object at 0x7e9c5b9b9a10> cannot be saved because the input shape is not available. Please specify an input shape either by calling `build(input_shape)` directly, or by calling the model on actual data using `Model()`, `Model.fit()`, or `Model.predict()`.

In [85]:
best_model = tf.keras.models.load_model("./best_hyperband")

In [86]:
best_model.evaluate(X_val, y_val)

35/35 [==============================] - 0s 2ms/step - loss: 0.1764 - msle: 0.1764


[0.17636224627494812, 0.17636224627494812]

In [10]:
for layer in best_model.layers:
    print(layer.output_shape)

AttributeError: The layer "flatten" has never been called and thus has no defined output shape.

Epoch 1/80
341/341 [==============================] - 3s 2ms/step - loss: 0.0728
Epoch 2/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0718
Epoch 3/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0726
Epoch 4/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0721
Epoch 5/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0714
Epoch 6/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0701
Epoch 7/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0704
Epoch 8/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0696
Epoch 9/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0696
Epoch 10/80
341/341 [==============================] - 1s 3ms/step - loss: 0.0696
Epoch 11/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0688
Epoch 12/80
341/341 [==============================] - 1s 2ms/step - loss: 0.0683
Epoch 13/80
341/341 [====

In [14]:
from sklearn.model_selection import train_test_split

X = df_test.copy()

X['datetime'] = pd.to_datetime(X['datetime'])
X['hour'] = X['datetime'].dt.hour
X['weekday'] = X['datetime'].dt.weekday
#X['month'] = X['datetime'].dt.month
X['year'] = X['datetime'].dt.month

# "count" es la columna objetivo, "casual" y "registered" son parte del objetivo
# pero solo queremos predecir la cantidad total de bicicletas alquiladas.
X = X.drop(columns=["datetime"])

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X)
X = norm_layer(X)

In [15]:
y = pd.DataFrame()
y["datetime"] = df_test["datetime"]
y["count"] = np.round(best_model.predict(X)).astype(int)

203/203 [==============================] - 0s 1ms/step


In [16]:
y.to_csv("submission.csv", index=False)

In [13]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.3 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 4.7 MB/s eta 0:00:0020.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.4 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.0/620.0 kB 12.8 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01


In [14]:
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])

def objective(trial):
    n_hidden = trial.suggest_int("n_hidden", 0, 8)
    n_neurons = trial.suggest_int("n_neurons", 16, 50)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    
    model = tf.keras.Sequential()
    model.add(norm_layer)
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(loss="msle", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=["msle"])
    model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
    score = model.evaluate(X_val, y_val, verbose=0)

    return score[1]    

In [16]:
import optuna

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-16 21:42:57,166] A new study created in memory with name: no-name-6cc92772-7a7a-41c2-a3f6-c185c6e91c2e


Epoch 1/5
307/307 [==============================] - 2s 3ms/step - loss: 2.0212 - msle: 2.0212 - val_loss: 2.0797 - val_msle: 2.0797
Epoch 2/5
307/307 [==============================] - 1s 2ms/step - loss: 1.8948 - msle: 1.8948 - val_loss: 1.6503 - val_msle: 1.6503
Epoch 3/5
307/307 [==============================] - 1s 2ms/step - loss: 1.2561 - msle: 1.2561 - val_loss: 1.1400 - val_msle: 1.1400
Epoch 4/5
307/307 [==============================] - 1s 2ms/step - loss: 1.1287 - msle: 1.1287 - val_loss: 1.0614 - val_msle: 1.0614
Epoch 5/5
307/307 [==============================] - 1s 2ms/step - loss: 1.0886 - msle: 1.0886 - val_loss: 1.0284 - val_msle: 1.0284


[I 2024-04-16 21:43:01,982] Trial 0 finished with value: 1.028429627418518 and parameters: {'n_hidden': 7, 'n_neurons': 19, 'learning_rate': 0.0008042722954183394}. Best is trial 0 with value: 1.028429627418518.


Epoch 1/5
307/307 [==============================] - 1s 2ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 2/5
307/307 [==============================] - 0s 1ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 3/5
307/307 [==============================] - 0s 1ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 4/5
307/307 [==============================] - 0s 1ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 5/5
307/307 [==============================] - 1s 2ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108


[I 2024-04-16 21:43:04,972] Trial 1 finished with value: 22.910757064819336 and parameters: {'n_hidden': 1, 'n_neurons': 29, 'learning_rate': 0.00037760516612011163}. Best is trial 1 with value: 22.910757064819336.


...

Epoch 1/5
307/307 [==============================] - 1s 2ms/step - loss: 1.9014 - msle: 1.9014 - val_loss: 1.8960 - val_msle: 1.8960
Epoch 2/5
307/307 [==============================] - 0s 2ms/step - loss: 1.7389 - msle: 1.7389 - val_loss: 1.7022 - val_msle: 1.7022
Epoch 3/5
307/307 [==============================] - 0s 2ms/step - loss: 1.5695 - msle: 1.5695 - val_loss: 1.5215 - val_msle: 1.5215
Epoch 4/5
307/307 [==============================] - 0s 2ms/step - loss: 1.4547 - msle: 1.4547 - val_loss: 1.4351 - val_msle: 1.4351
Epoch 5/5
307/307 [==============================] - 0s 2ms/step - loss: 1.3897 - msle: 1.3897 - val_loss: 1.3463 - val_msle: 1.3463


[I 2024-04-16 21:48:56,166] Trial 98 finished with value: 1.3462544679641724 and parameters: {'n_hidden': 1, 'n_neurons': 19, 'learning_rate': 0.000801913068246844}. Best is trial 1 with value: 22.910757064819336.


Epoch 1/5
307/307 [==============================] - 2s 2ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 2/5
307/307 [==============================] - 1s 2ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 3/5
307/307 [==============================] - 1s 2ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 4/5
307/307 [==============================] - 1s 2ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108
Epoch 5/5
307/307 [==============================] - 1s 2ms/step - loss: 23.1158 - msle: 23.1158 - val_loss: 22.9108 - val_msle: 22.9108


[I 2024-04-16 21:49:00,279] Trial 99 finished with value: 22.910757064819336 and parameters: {'n_hidden': 5, 'n_neurons': 25, 'learning_rate': 0.00018560432398704026}. Best is trial 1 with value: 22.910757064819336.


Number of finished trials: 100
Best trial:
  Value: 22.910757064819336
  Params: 
    n_hidden: 1
    n_neurons: 29
    learning_rate: 0.00037760516612011163


## Parte 5 - Pipeline



*   Incorporar el estimador con mejor desempeño a un *pipeline* similar al implementado en el taller 3. Puede ser útil la librería [scikeras](https://adriangb.com/scikeras/stable/migration.html).
*   Subir los resultados de los datos *test* a la página de la competencia.



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

categorical_cols = ["season", "holiday", "workingday", "weather", "hour", "weekday", "month", "year"]
numerical_cols = ["temp", "atemp", "humidty", "windspeed"]

preprocess = Pipeline([
    ('encoding', 
])

## Parte 6 - Shapley Values

- Siguiendo el [ejemplo](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html) presentado en la sección *Explaining a non-additive boosted tree model* de la librería *shap*. Encuentre los shapley values para el mejor modelo que haya encontrado.
- Elija una muestra y mediante la función *shap.plots.waterfall* observe la explicación de la predicción del modelo.

Más explicación de los Shapley Values y su uso en la interpretabilidad de modelos de aprendizaje automático se puede encontrar [aquí](https://christophm.github.io/interpretable-ml-book/shapley.html).